# DBMI SET Effort Analysis - Data Exploration - Combined Monday.com and Toggl.com Data

In [ ]:
import pathlib
import zipfile

import duckdb
import pandas as pd

In [ ]:
# pull data from dvc remote
!dvc pull

In [ ]:
# setting some paths for later reference
monday_data = "../data/monday.com/dbmi-set-monday-acct-project-tasks.parquet"
toggl_data = list(
    pathlib.Path("../data/toggl.com/").glob("DB-Toggl_Track_summary_report_*.csv")
)

In [ ]:
# read monday data into df
df_monday = pd.read_parquet(monday_data)
print(df_monday.info())
df_monday.head()

In [ ]:
df_monday["Acct_Name"].unique()

In [ ]:
# read toggl data into df
df_toggl = pd.concat([pd.read_csv(filepath) for filepath in toggl_data])
print(df_toggl.info())
df_toggl.head()

In [ ]:
df_toggl["Client"].unique()

In [ ]:
# rename columns for join clarity
df_monday = df_monday.rename(
    columns={"Acct_Name": "Client", "Task_Actual Time": "Duration"}
)
df_toggl = df_toggl.rename(columns={"Description": "Task_Name"})

In [ ]:
# relabel clients from monday for dataset clarity
df_monday["Client"] = (
    df_monday["Client"]
    .str.replace("HealthAI: Way Lab", "Way Lab")
    .replace("HealthAI: Admin & Operations", "DBMI Administration")
)

In [ ]:
column_subset = ["Client", "Task_Name", "Duration"]
df_combined = pd.concat([df_monday[column_subset], df_toggl[column_subset]]).dropna()
df_combined.head()

In [ ]:
# cast duration as a datetime and as separate minutes column
df_combined["Duration"] = pd.to_timedelta(df_combined["Duration"], errors="ignore")
df_combined["Duration_Minutes"] = (
    df_combined["Duration"].dt.total_seconds().div(60).astype(int)
)

In [ ]:
df_combined.groupby("Client")["Duration_Minutes"].sum().sort_values().plot(kind="barh")

In [ ]:
# export for potential later use
df_combined.to_parquet("../data/analysis/monday_and_toggl_task_analysis.parquet")

In [ ]:
!dvc add ../data/analysis/monday_and_toggl_task_analysis.parquet
!git add ../data/analysis/monday_and_toggl_task_analysis.parquet.dvc
!dvc push

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client = 'Way Lab' AND
    (
        df_combined.Task_Name ILIKE ('%PR%Review%')
        OR df_combined.Task_Name ILIKE ('%Review%PR%')
        OR df_combined.Task_Name ILIKE ('%PR''s%')
    )

"""
result = duckdb.sql(sql_stmt).df()
result.info()
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client = 'Way Lab' AND
    (
        df_combined.Task_Name ILIKE ('%PR%Review%')
        OR df_combined.Task_Name ILIKE ('%Review%PR%')
        OR df_combined.Task_Name ILIKE ('%PR''s%')
    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client = 'Way Lab' AND
    (
        df_combined.Task_Name ILIKE ('%pycytominer-transform%')
        OR df_combined.Task_Name ILIKE ('%cytotable%')

    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client = 'Way Lab' AND
    (
        df_combined.Task_Name ILIKE ('%pycytominer%')
        and not df_combined.Task_Name ILIKE ('%pycytominer-transform%')
        and not df_combined.Task_Name ILIKE ('%pycytominer-performance%')
        and not df_combined.Task_Name ILIKE ('%cytotable%')

    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client = 'DBMI Administration' AND
    (
        df_combined.Task_Name ILIKE ('%totw%')
        or df_combined.Task_Name ILIKE ('%blog%')
        

    ) AND NOT df_combined.Task_Name ILIKE ('%bssw%')
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration', 'Way Lab')
    AND
    (
        (df_combined.Task_Name ILIKE ('%bssw%')
        and df_combined.Task_Name ILIKE ('%blog%'))
        OR
        (df_combined.Task_Name ILIKE ('%bssw%abstract%'))
    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration', 'Way Lab')
    AND
    (
        df_combined.Task_Name ILIKE ('%sqlite-clean%')
        OR df_combined.Task_Name ILIKE ('%sqlite%clean%')
        OR df_combined.Task_Name ILIKE ('%pycytominer%performance%')
    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
# result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration', 'Way Lab')
    AND
    (
        df_combined.Task_Name ILIKE ('%bssw%application%')
        OR df_combined.Task_Name ILIKE ('%bssw%grant%')
    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration', 'Way Lab')
    AND
    (
        df_combined.Task_Name ILIKE ('%scipy%')
        OR df_combined.Task_Name ILIKE ('%data grammar%')
        OR df_combined.Task_Name ILIKE ('%Conference topic submission%')
    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration')
    AND
    (
        df_combined.Task_Name ILIKE ('%ospo%')
        or df_combined.Task_Name ILIKE ('%sloan%')
        or df_combined.Task_Name ILIKE ('%ossr%')

    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration')
    AND
    (
        df_combined.Task_Name ILIKE ('%effort%')
        OR df_combined.Task_Name ILIKE ('%toggl%')

    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('Way Lab')
    AND
    (
        df_combined.Task_Name ILIKE ('%assay%')
        OR df_combined.Task_Name ILIKE ('%bucket%')

    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration')
    AND
    (
        df_combined.Task_Name ILIKE ('%cloud%storage%')
        OR df_combined.Task_Name ILIKE ('%bucket%')
        OR df_combined.Task_Name ILIKE ('%google%')

    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
# result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('DBMI Administration')
    AND
    (
        df_combined.Task_Name ILIKE ('%cloud%storage%')
    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result
result["Duration_Minutes"].sum() / 60

In [ ]:
sql_stmt = """
SELECT * FROM df_combined
WHERE df_combined.Client in ('Way Lab')
AND ( 
        df_combined.Task_Name ILIKE ('%hiring%')
        OR df_combined.Task_Name ILIKE ('%pra%')
        OR df_combined.Task_Name ILIKE ('%interview%')
    )
"""
result = duckdb.sql(sql_stmt).df()
result.info()
result["Task_Name"].tolist()
result["Duration_Minutes"].sum() / 60